In [1]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [3, 5, 10, 15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [2]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [3]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
# print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_perf_ratios.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-04
DEST_DIR: ..\data

source_path: ..\data\2025-04-04_df_OHLCV_clean.parquet
dest_path: ..\data\2025-04-04_df_perf_ratios.parquet


In [4]:
import pandas as pd

# Get tickers from df_finviz.pkl file
df_finviz = pd.read_parquet(f'..\data\{date_str}_df_finviz.parquet')
tickers = df_finviz.index.to_list()

In [ ]:
import pandas as pd

pd.set_option('display.width', 1000) 

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# # Display the first few rows of the DataFrame to verify
# df = df[['Adj Close']].copy()
display(df)

Open    High     Low   Close  Adj Close   Volume  Adj Open  Adj High  Adj Low
Symbol Date                                                                                       
CLX    2025-04-04  148.72  150.84  143.37  143.86     143.86  2083767    148.72    150.84   143.37
       2025-04-03  148.46  150.60  148.04  148.69     148.69  1199500    148.46    150.60   148.04
       2025-04-02  148.05  148.34  146.05  147.62     147.62   813000    148.05    148.34   146.05
       2025-04-01  148.08  148.08  146.51  147.91     147.91  1080800    148.08    148.08   146.51
       2025-03-31  146.23  148.48  146.23  147.25     147.25  1201800    146.23    148.48   146.23
...                   ...     ...     ...     ...        ...      ...       ...       ...      ...
COOP   2024-04-11   76.55   77.29   76.03   77.22      77.22   279700     76.55     77.29    76.03
       2024-04-10   75.83   77.72   75.32   76.45      76.45   498800     75.83     77.72    75.32
       2024-04-09   78.51   78.54   76.25   77.57      77.57   245600     78.51     78.54    76.25
       2024-04-08   77.68   78.15   77.06   77.54      77.54   463800     77.68     78.15    77.06
       2024-04-05   75.92   77.53   75.53   77.13      77.13   602300     75.92     77.53    75.53

[386038 rows x 9 columns]

In [ ]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
CLX    2025-04-04     143.86
       2025-04-03     148.69
       2025-04-02     147.62
       2025-04-01     147.91
       2025-03-31     147.25
...                      ...
COOP   2024-04-11      77.22
       2024-04-10      76.45
       2024-04-09      77.57
       2024-04-08      77.54
       2024-04-05      77.13

[386038 rows x 1 columns]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 386038 entries, ('CLX', Timestamp('2025-04-04 00:00:00')) to ('COOP', Timestamp('2024-04-05 00:00:00'))
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Close  386038 non-null  float64
dtypes: float64(1)
memory usage: 4.5+ MB


In [8]:
import pandas as pd

def select_tickers_data(df, tickers):
  """
  Selects data for a list of tickers from a DataFrame with a MultiIndex
  where the first level is the ticker and the second level is the date.

  Args:
    df (pd.DataFrame): The input DataFrame with a MultiIndex.
    tickers (list): A list of ticker symbols to select.

  Returns:
    pd.DataFrame: A DataFrame containing only the data for the specified tickers.
                  Rows for tickers not found will not be included.
  """
  valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

  if not valid_tickers:
    print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
    return pd.DataFrame()

  try:
    ticker_df = df.loc[valid_tickers]
    return ticker_df
  except KeyError as e:
    print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
    return pd.DataFrame()


# Example usage:
# ticker_list = ["UBS", "AAPL", "MSFT", "GEV"]  # Include some valid and invalid tickers
selected_data = select_tickers_data(df, tickers)
print(selected_data)

                   Adj Close
Symbol Date                 
AAPL   2025-04-04     188.38
       2025-04-03     203.19
       2025-04-02     223.89
       2025-04-01     223.19
       2025-03-31     222.13
...                      ...
BKLC   2024-04-11      97.27
       2024-04-10      96.59
       2024-04-09      97.42
       2024-04-08      97.32
       2024-04-05      97.42

[386038 rows x 1 columns]


In [9]:
# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

import sys

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

import utils

list_dfs = utils.get_latest_dfs(selected_data, DAYS_RATIO)

In [10]:
import numpy as np

all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        # Suppress division warnings just for this calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
       Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A    -204.991083  -15.850762       0.0 -13.576523  -11.371100  0.121459  -11.415996    -9.803333   0.104632   -8.540250    -7.937587   0.182221   -6.498826    -6.608261   0.307531   -3.860204    -4.453333   0.507592    -2.471881     -3.073085    0.652861    -1.217535     -1.570256    0.808377
AA   -107.194076  -15.788181       0.0 -11.923817  -10.528191  0.105663  -11.106768    -9.568575   0.094825   -9.083853    -8.420301   0.175879   -4.636415    -5.086422   0.454572   -2.725191    -3.169771   0.627061    -2.014561     -2.463068    0.706392    -0.613821     -0.831460    0.903286
AAL   -12.482107  -11.803624       0.0  -7.435087   -7.815331  0.213042  -10.

In [11]:
import numpy as np
import pandas as pd

def find_nan_inf(df):
    """
    Finds locations of NaN, Inf, and -Inf values in a DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame to check
        
    Returns:
        dict: Dictionary with column names as keys and dicts of 
              {type: indices} as values
    """
    nan_inf_locations = {}
    for col in df.columns:
        locations = {
            'NaN': df[col].index[df[col].isnull()].tolist(),
            'Inf': df[col].index[df[col] == np.inf].tolist(),
            '-Inf': df[col].index[df[col] == -np.inf].tolist()
        }
        if any(locations.values()):
            nan_inf_locations[col] = locations
    return nan_inf_locations

def print_nan_inf_locations(locations, message):
    """Prints NaN/Inf locations with a descriptive message."""
    print(message)
    if locations:
        for col, locs in locations.items():
            print(f"Column: {col}")
            for typ, indices in locs.items():
                if indices:
                    print(f"  {typ}: {indices}")
    else:
        print("No NaN or Inf values found.")

def get_column_replacement_values(df):
    """
    Calculate replacement values for each column (max for Inf/NaN, min for -Inf).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        tuple: (max_values, min_values) dictionaries with column replacement values
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    max_values = {}
    min_values = {}
    
    for col in numeric_cols:
        # Get finite values only
        finite_vals = df[col][np.isfinite(df[col])]
        if len(finite_vals) > 0:
            max_values[col] = finite_vals.max()
            min_values[col] = finite_vals.min()
        else:
            # If no finite values, use default
            max_values[col] = 1e5
            min_values[col] = -1e5
    
    return max_values, min_values

# Check for NaN or Inf values
has_nan_inf = combined_df.isnull().any().any() or not np.isfinite(combined_df).all().all()
print(f'combined_df has_nan_inf: {has_nan_inf}')

if has_nan_inf:
    # Calculate replacement values for each column
    max_values, min_values = get_column_replacement_values(combined_df)
    print("Replacing NaN/Inf values with column-specific max/min values")
    
    nan_inf_locations_before = find_nan_inf(combined_df)
    print_nan_inf_locations(nan_inf_locations_before, "NaN/Inf Locations BEFORE Replacement:")
    
    # Create cleaned DataFrame
    df_to_save = combined_df.copy()
    
    # Replace values column by column
    for col in df_to_save.columns:
        if col in max_values:  # Only numeric columns will be in max_values
            # Replace NaN and +Inf with max value
            df_to_save[col] = df_to_save[col].fillna(max_values[col])
            df_to_save[col] = df_to_save[col].replace([np.inf], max_values[col])
            # Replace -Inf with min value
            df_to_save[col] = df_to_save[col].replace([-np.inf], min_values[col])
else:
    print("No NaN or Inf values found.")
    df_to_save = combined_df

# Verification
print(f'df_to_save has_nan_inf: {df_to_save.isnull().any().any() or not np.isfinite(df_to_save).all().all()}')
print_nan_inf_locations(find_nan_inf(df_to_save), "NaN/Inf Locations AFTER Replacement:")

# Output cleaned DataFrame
print(df_to_save)

combined_df has_nan_inf: True
Replacing NaN/Inf values with column-specific max/min values
NaN/Inf Locations BEFORE Replacement:
Column: Sharpe 3d
  -Inf: ['IBDQ']
Column: Sortino 3d
  Inf: ['AGG', 'BCE', 'BILS', 'BIV', 'BLV', 'BND', 'BNDX', 'BSV', 'CMF', 'EAGG', 'EDV', 'FIXD', 'GBIL', 'GOVT', 'GVI', 'IAGG', 'ICSH', 'IEF', 'IEI', 'JCPB', 'JMBS', 'LMBS', 'SCHO', 'SCHR', 'SGOV', 'SHM', 'SPAB', 'SPMB', 'SPTI', 'SPTL', 'SPTS', 'SUB', 'TFI', 'TFLO', 'TLH', 'TLT', 'TMF', 'TOTL', 'VGIT', 'VGLT', 'VGSH', 'VMBS', 'VTEB']
Column: Omega 3d
  NaN: ['AGG', 'BCE', 'BILS', 'BIV', 'BLV', 'BND', 'BNDX', 'BSV', 'CMF', 'EAGG', 'EDV', 'FIXD', 'GBIL', 'GOVT', 'GVI', 'IAGG', 'ICSH', 'IEF', 'IEI', 'JCPB', 'JMBS', 'LMBS', 'SCHO', 'SCHR', 'SGOV', 'SHM', 'SPAB', 'SPMB', 'SPTI', 'SPTL', 'SPTS', 'SUB', 'TFI', 'TFLO', 'TLH', 'TLT', 'TMF', 'TOTL', 'VGIT', 'VGLT', 'VGSH', 'VMBS', 'VTEB']
Column: Sortino 5d
  Inf: ['BLV', 'IAGG', 'SUB']
Column: Omega 5d
  NaN: ['BLV', 'IAGG', 'SUB']
Column: Sortino 10d
  Inf: ['IAGG'

In [12]:
ticker_w_inf = nan_inf_locations_before['Sortino 3d']['Inf'][0]
ticker_w_inf 

'AGG'

#### Use Cell below to check the inf, -inf and NaN replacement

In [13]:
print(f"combined_df.loc['ticker_w_inf']")
display(combined_df.loc[ticker_w_inf])
print(f"\ndf_to_save.loc['ticker_w_inf']")
display(df_to_save.loc[ticker_w_inf])

combined_df.loc['ticker_w_inf']


Sharpe 3d       15.075804
Sortino 3d            inf
Omega 3d              NaN
Sharpe 5d       11.298085
Sortino 5d      50.429633
Omega 5d         7.353537
Sharpe 10d       8.060198
Sortino 10d     23.121192
Omega 10d        3.859694
Sharpe 15d       4.773870
Sortino 15d      9.575133
Omega 15d        2.177501
Sharpe 30d       2.701479
Sortino 30d      4.818149
Omega 30d        1.500462
Sharpe 60d       2.660260
Sortino 60d      4.601861
Omega 60d        1.512313
Sharpe 120d      0.020505
Sortino 120d     0.029907
Omega 120d       1.003323
Sharpe 250d      0.612976
Sortino 250d     0.885469
Omega 250d       1.102828
Name: AGG, dtype: float64


df_to_save.loc['ticker_w_inf']


Sharpe 3d         15.075804
Sortino 3d      1413.904057
Omega 3d         126.960585
Sharpe 5d         11.298085
Sortino 5d        50.429633
Omega 5d           7.353537
Sharpe 10d         8.060198
Sortino 10d       23.121192
Omega 10d          3.859694
Sharpe 15d         4.773870
Sortino 15d        9.575133
Omega 15d          2.177501
Sharpe 30d         2.701479
Sortino 30d        4.818149
Omega 30d          1.500462
Sharpe 60d         2.660260
Sortino 60d        4.601861
Omega 60d          1.512313
Sharpe 120d        0.020505
Sortino 120d       0.029907
Omega 120d         1.003323
Sharpe 250d        0.612976
Sortino 250d       0.885469
Omega 250d         1.102828
Name: AGG, dtype: float64

In [ ]:
# Save the DataFrame to parquet file (always do this)
df_to_save.to_parquet(dest_path, engine='pyarrow', compression='zstd')
print(f"DataFrame successfully saved to {dest_path}")



DataFrame successfully saved to ..\data\2025-04-04_df_perf_ratios.parquet


In [15]:
import pandas as pd
import numpy as np
from tabulate import tabulate

_symbols = ['USFR','AAPL', 'MSFT', 'GOOG', 'NVDA', 'IBIT', 'GLD', 'VCIT']
# Create and display the formatted table
selected_df = df_to_save.loc[_symbols] #Use df_to_save here
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+-------------+--------------+------------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 3d |   Sortino 3d |   Omega 3d |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============+===============+====